<a href="https://colab.research.google.com/github/EfaOnyianta/Sales-Prediction/blob/main/Project_part_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Loading Data

In [ ]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.compose import make_column_selector, make_column_transformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, \
OrdinalEncoder
from sklearn.pipeline import make_pipeline
from sklearn.impute import SimpleImputer

from sklearn import set_config
set_config(display='diagram')


In [ ]:
path = '/content/sales_predictions (1).csv'
df = pd.read_csv('/content/sales_predictions (1).csv')
df.head()

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
0,FDA15,9.30,Low Fat,0.016047,Dairy,249.8092,OUT049,1999,Medium,Tier 1,Supermarket Type1,3735.1380
1,DRC01,5.92,Regular,0.019278,Soft Drinks,48.2692,OUT018,2009,Medium,Tier 3,Supermarket Type2,443.4228
2,FDN15,17.50,Low Fat,0.016760,Meat,141.6180,OUT049,1999,Medium,Tier 1,Supermarket Type1,2097.2700
3,FDX07,19.20,Regular,0.000000,Fruits and Vegetables,182.0950,OUT010,1998,NaN,Tier 3,Grocery Store,732.3800
4,NCD19,8.93,Low Fat,0.000000,Household,53.8614,OUT013,1987,High,Tier 3,Supermarket Type1,994.7052


we need to make a copy of the original to avoid manipulations

In [ ]:
#SAVE copy
sales_df = df.copy()

##Checking for duplicates and missing Values, and dropping dupes and replacing missing values 

In [ ]:
#get sum of missing values from te df
sales_df.isna().sum().sum()

3873

In [ ]:
#display the sum of missing values from each column
sales_df.isna().sum()

Item_Identifier                 0
Item_Weight                  1463
Item_Fat_Content                0
Item_Visibility                 0
Item_Type                       0
Item_MRP                        0
Outlet_Identifier               0
Outlet_Establishment_Year       0
Outlet_Size                  2410
Outlet_Location_Type            0
Outlet_Type                     0
Item_Outlet_Sales               0
dtype: int64

In [ ]:
#check for duplicate rows
sales_df.duplicated().sum()
# no duplicated rows no need to drop anythything


0

In [ ]:
#analyse data info from the data
sales_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8523 entries, 0 to 8522
Data columns (total 12 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Item_Identifier            8523 non-null   object 
 1   Item_Weight                7060 non-null   float64
 2   Item_Fat_Content           8523 non-null   object 
 3   Item_Visibility            8523 non-null   float64
 4   Item_Type                  8523 non-null   object 
 5   Item_MRP                   8523 non-null   float64
 6   Outlet_Identifier          8523 non-null   object 
 7   Outlet_Establishment_Year  8523 non-null   int64  
 8   Outlet_Size                6113 non-null   object 
 9   Outlet_Location_Type       8523 non-null   object 
 10  Outlet_Type                8523 non-null   object 
 11  Item_Outlet_Sales          8523 non-null   float64
dtypes: float64(4), int64(1), object(7)
memory usage: 799.2+ KB


In [ ]:
#display descriptive stats for categorical columns 
sales_df.describe(include = 'object')

,Item_Identifier,Item_Fat_Content,Item_Type,Outlet_Identifier,Outlet_Size,Outlet_Location_Type,Outlet_Type
count,8523,8523,8523,8523,6113,8523,8523
unique,1559,5,16,10,3,3,4
top,FDW13,Low Fat,Fruits and Vegetables,OUT027,Medium,Tier 3,Supermarket Type1
freq,10,5089,1232,935,2793,3350,5577


I think ill get rid of item identifier, there are too many variables, vs the others might be predictable/ add to the data. 

In [ ]:
#now im going to check the values in the columns that are left
sales_df.value_counts('Item_Fat_Content')

Item_Fat_Content
Low Fat    5089
Regular    2889
LF          316
reg         117
low fat     112
dtype: int64

So there are incorrect values, this column is suppost to only have 2 values Regular and Low Fat, im going to replace LF, reg, and low fat. 

In [ ]:
sales_df['Item_Fat_Content'] = sales_df['Item_Fat_Content']. replace({'LF':'Low Fat','reg': 'Regular','low fat': 'Low Fat'})
sales_df.value_counts('Item_Fat_Content')
#The duplicate values have been changed. 

Item_Fat_Content
Low Fat    5517
Regular    3006
dtype: int64

In [ ]:
sales_df.value_counts('Outlet_Type')
#the other vluaes were checked and there were no other dupl values. 

Outlet_Type
Supermarket Type1    5577
Grocery Store        1083
Supermarket Type3     935
Supermarket Type2     928
dtype: int64

In [ ]:
sales_df.value_counts('Outlet_Size')
#Hight doesnt make sense when it comes to outlet size, im replacing it to large. 

Outlet_Size
Medium    2793
Small     2388
High       932
dtype: int64

In [ ]:
sales_df['Outlet_Size'] = sales_df['Outlet_Size']. replace('High','Large')
sales_df.value_counts('Outlet_Size')

Outlet_Size
Medium    2793
Small     2388
Large      932
dtype: int64

##Train test splitting and preprocessing 

I'm going to Identify the features (X) and target (y)  "Item_Outlet_Sales" column is our target and the rest of the relevant variables are out features. i already identified the categorical columns and decided which ones i want to keep. ill review the numberical to see if there are any columns that should be not included in the data split. 

In [ ]:
#I'm going to Identify the features (X) and target (y)  "Item_Outlet_Sales" column is our target and the rest of the relevant variables are out features
sales_df.head().value_counts
sales_df.describe(include = 'float')

,Item_Weight,Item_Visibility,Item_MRP,Item_Outlet_Sales
count,7060.000000,8523.000000,8523.000000,8523.000000
mean,12.857645,0.066132,140.992782,2181.288914
std,4.643456,0.051598,62.275067,1706.499616
min,4.555000,0.000000,31.290000,33.290000
25%,8.773750,0.026989,93.826500,834.247400
50%,12.600000,0.053931,143.012800,1794.331000
75%,16.850000,0.094585,185.643700,3101.296400
max,21.350000,0.328391,266.888400,13086.964800


In [ ]:
#splitting data
target= 'Item_Outlet_Sales'
X = sales_df.drop(columns=[target,'Item_Identifier','Outlet_Establishment_Year']).copy()
y =  sales_df[target].copy()
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [ ]:
#making numeric and categorical selectors
category_selector = make_column_selector(dtype_include='object')
numb_selector = make_column_selector(dtype_include='number')
numb_columns = numb_selector(X_train)
category_columns = category_selector(X_train)
numb_columns
category_columns

['Item_Fat_Content',
 'Item_Type',
 'Outlet_Identifier',
 'Outlet_Size',
 'Outlet_Location_Type',
 'Outlet_Type']

In [ ]:
#isolating numeric columsn to see whats missing values/ data 
df_numb= df[numb_columns]
df_numb.loc[:, df_numb.isna().any()]
#item weight is the numberic column missing data and it seems to be floats so the mean strategy can be used 

,Item_Weight
0,9.300
1,5.920
2,17.500
3,19.200
4,8.930
...,...
8518,6.865
8519,8.380
8520,10.600
8521,7.210


In [ ]:
# creating imputer for the mean stategy and constant for missing data and scaler and one hot encoder
mean_imputer = SimpleImputer(strategy='mean')
missing_imputer = SimpleImputer( strategy = 'constant')
scaler = StandardScaler()
ohe = OneHotEncoder(handle_unknown='ignore', sparse=False)



In [ ]:
# making numeric pipeline
numeric_pipe = make_pipeline(mean_imputer, scaler)
numeric_pipe

Pipeline(steps=[('simpleimputer', SimpleImputer()),
                ('standardscaler', StandardScaler())])

In [ ]:
# categorical pipeline
categorical_pipe = make_pipeline(missing_imputer, ohe)
categorical_pipe

Pipeline(steps=[('simpleimputer', SimpleImputer(strategy='constant')),
                ('onehotencoder',
                 OneHotEncoder(handle_unknown='ignore', sparse=False))])

In [ ]:
# were going to make tuples for each data type and the pipeline above and make a column transformer
number_tuple = (numeric_pipe, numb_selector)
category_tuple = (categorical_pipe, category_selector)
preprocessor = make_column_transformer(number_tuple, category_tuple)
preprocessor

ColumnTransformer(transformers=[('pipeline-1',
                                 Pipeline(steps=[('simpleimputer',
                                                  SimpleImputer()),
                                                 ('standardscaler',
                                                  StandardScaler())]),
                                 <sklearn.compose._column_transformer.make_column_selector object at 0x7ff612e04ca0>),
                                ('pipeline-2',
                                 Pipeline(steps=[('simpleimputer',
                                                  SimpleImputer(strategy='constant')),
                                                 ('onehotencoder',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse=False))]),
                                 <sklearn.compose._column_transformer.make_column_selector object at 0x7ff612e04d60>)])

##Transforming the data ande inspecting

In [ ]:
# were going to fit the above column transformer to the training data (never testing)
preprocessor.fit(X_train)

ColumnTransformer(transformers=[('pipeline-1',
                                 Pipeline(steps=[('simpleimputer',
                                                  SimpleImputer()),
                                                 ('standardscaler',
                                                  StandardScaler())]),
                                 <sklearn.compose._column_transformer.make_column_selector object at 0x7ff612e04ca0>),
                                ('pipeline-2',
                                 Pipeline(steps=[('simpleimputer',
                                                  SimpleImputer(strategy='constant')),
                                                 ('onehotencoder',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse=False))]),
                                 <sklearn.compose._column_transformer.make_column_selector object at 0x7ff612e04d60>)])

In [ ]:
# and now were going to transform all the data 
X_train_processed = preprocessor.transform(X_train)
X_test_processed = preprocessor.transform(X_test)

In [ ]:
np.isnan(X_train_processed).sum().sum()
# no missing values in the training data 

0

In [ ]:
np.isnan(X_test_processed).sum().sum()
# no missing data in the training data 

0

In [ ]:
X_train_processed.dtype
# the training data in train processed are all floats 

dtype('float64')

In [ ]:
X_test_processed.dtype
# the data in the test processed are floats as well

dtype('float64')

In [ ]:
X_train_processed.shape

(6392, 42)

In [ ]:
X_train_processed
# my numpy array :)

array([[ 0.81724868, -0.71277507,  1.82810922, ...,  0.        ,
         1.        ,  0.        ],
       [ 0.5563395 , -1.29105225,  0.60336888, ...,  0.        ,
         1.        ,  0.        ],
       [-0.13151196,  1.81331864,  0.24454056, ...,  1.        ,
         0.        ,  0.        ],
       ...,
       [ 1.11373638, -0.92052713,  1.52302674, ...,  1.        ,
         0.        ,  0.        ],
       [ 1.76600931, -0.2277552 , -0.38377708, ...,  1.        ,
         0.        ,  0.        ],
       [ 0.81724868, -0.95867683, -0.73836105, ...,  1.        ,
         0.        ,  0.        ]])